In [1]:
!pip install chromadb sentence-transformers pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.4 MB/s eta 0

In [3]:
import os
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import chromadb

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
client = chromadb.PersistentClient(path="./chroma_emergency_db")
collection = client.get_or_create_collection("medical_papers")

pdf_dir = "pdfs"  # 코랩에 업로드한 폴더명
for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):
        reader = PdfReader(os.path.join(pdf_dir, filename))
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
        # 문단 기준 쪼개기 (500자 이상만)
        paragraphs = [p.strip() for p in text.split('\n\n') if len(p.strip()) > 100]
        for i, para in enumerate(paragraphs):
            doc_id = f"{filename}_{i}"
            emb = embedding_model.encode(para)
            collection.add(
                documents=[para],
                metadatas=[{"source": filename, "para_id": i}],
                ids=[doc_id],
                embeddings=[emb]
            )
print("ChromaDB 구축 완료!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ChromaDB 구축 완료!


In [4]:
!zip -r chroma_emergency_db.zip chroma_emergency_db
from google.colab import files
files.download("chroma_emergency_db.zip")

  adding: chroma_emergency_db/ (stored 0%)
  adding: chroma_emergency_db/chroma.sqlite3 (deflated 49%)
  adding: chroma_emergency_db/698d58c0-8a7b-455c-8bcb-a8192a61618d/ (stored 0%)
  adding: chroma_emergency_db/698d58c0-8a7b-455c-8bcb-a8192a61618d/data_level0.bin (deflated 100%)
  adding: chroma_emergency_db/698d58c0-8a7b-455c-8bcb-a8192a61618d/header.bin (deflated 61%)
  adding: chroma_emergency_db/698d58c0-8a7b-455c-8bcb-a8192a61618d/link_lists.bin (stored 0%)
  adding: chroma_emergency_db/698d58c0-8a7b-455c-8bcb-a8192a61618d/length.bin (deflated 100%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
query = "고령, 흉통, 호흡곤란"
query_vec = embedding_model.encode(query)
results = collection.query(query_embeddings=[query_vec], n_results=3)
for doc in results['documents'][0]:
    print(doc)

271
� 원저 �대한응급의학회지 제 33 권 제 3 호
Volume 33, Number 3, June, 2022
AAddmmiinniissttrraattiioonn
응급실 전담 의사의 내과 입원 결정권과 응급 병동이 응급실 체류 시간에 미치는 영향
박준성∙장태창∙이숙희∙고승현∙이경우∙김균무∙서영우
대구가톨릭대학교 의과대학 응급의학교실
Effects of emergency doctor’s hospitalization authority from internal medicine 
and utilizing emergency ward on length of stay in emergency department
Jun Sung Park, Tae Chang Jang, Suk Hee Lee, Seung Hyun Ko, Kyung Woo Lee, Gyun Moo Kim, Young Woo Seo
Department of Emergency Medicine, Daegu Catholic University School of Medicine, Daegu, Korea
Objective: The emergency department (ED) length of stay (LOS) is related to ED overcrowding and emergency practice.
This study aimed to investigate the effects of enabling an ED doctor to have the authority to make hospitalization deci-
sions and utilization of the emergency ward on ED LOS.
Methods: This retrospective observational study included patients who were admitted through a local emergency med-
ical center. We compared the ED LOS between the periods' M